In [1]:
## module python permettant d'utiliser le protocole http
!pip install requests
## package is used to automate web browser interaction from Python
!pip install selenium==3.5.0
## library that makes it easy to scrape information from web pages
!pip install beautifulsoup4

  Using cached selenium-3.5.0-py2.py3-none-any.whl (921 kB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.0
    Uninstalling selenium-4.8.0:
      Successfully uninstalled selenium-4.8.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
undetected-chromedriver 3.2.1 requires selenium>=4.0.0, but you have selenium 3.5.0 which is incompatible.


In [4]:
## a list that conatins all the small case letters
atoz = []
for i in range(97,123):
    atoz.append(chr(i))

In [5]:
## all players url
url_All_Players = "https://www.basketball-reference.com/players/{}/"

In [8]:
## all players urls list
All_players_urls_list = []

In [206]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## to lunch the web chrome driver
driver = webdriver.Chrome(
    executable_path="./chromedriver.exe"
    )

In [207]:
## extract all the html pages that contains players urls
for letter in atoz:
    url = url_All_Players.format(letter)

    ## enter url
    driver.get(url)
    ## scroll down the page
    driver.execute_script("window.scrollTo(1,10000)")
    ## 1 sec pause (the website does not allow fast scrapping and detects automats)
    time.sleep(1)

    ## save html pages
    with open("atoz_pages/{}.html".format(letter), "w+", encoding="utf-8") as f:
        f.write(driver.page_source)

In [9]:
## extract each players url (we need the url to get his stats later)
from bs4 import BeautifulSoup
import re

## go through the html pages from a to z 
for letter in atoz:
    with open("atoz_pages/{}.html".format(letter), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    
    all_players = soup.find_all(id="players")[0]
    ## extract the players link
    for link, year_max, year_min in zip(all_players.find_all('a', href=re.compile(r'/players/[a-z]/[a-zA-Z0-9_]+\.html')), all_players.find_all("td", attrs={"data-stat": "year_max"}), all_players.find_all("td", attrs={"data-stat": "year_min"})):
        
        ## get the year when the player stopped playing
        y_max = year_max.text.strip()
        y_min = year_min.text.strip()
        ## test the year when the player stopped playing
        ## we remove all the players that didn't play from 1993 to 2024
        if int(y_max) >= 1993 and int(y_min) < 2024 :
            href = link['href']
            player_url = "https://www.basketball-reference.com" + href
            All_players_urls_list.append(player_url)
        

In [10]:
## all players urls active and inactive
All_players_urls_list

['https://www.basketball-reference.com/players/a/abdelal01.html',
 'https://www.basketball-reference.com/players/a/abdulma02.html',
 'https://www.basketball-reference.com/players/a/abdulta01.html',
 'https://www.basketball-reference.com/players/a/abdursh01.html',
 'https://www.basketball-reference.com/players/a/abrinal01.html',
 'https://www.basketball-reference.com/players/a/achiupr01.html',
 'https://www.basketball-reference.com/players/a/ackeral01.html',
 'https://www.basketball-reference.com/players/a/acresma01.html',
 'https://www.basketball-reference.com/players/a/acyqu01.html',
 'https://www.basketball-reference.com/players/a/adamsha01.html',
 'https://www.basketball-reference.com/players/a/adamsja01.html',
 'https://www.basketball-reference.com/players/a/adamsjo01.html',
 'https://www.basketball-reference.com/players/a/adamsmi01.html',
 'https://www.basketball-reference.com/players/a/adamsst01.html',
 'https://www.basketball-reference.com/players/a/addisra01.html',
 'https://ww

In [11]:
len(All_players_urls_list)

2789

In [16]:
listp = All_players_urls_list[2355:2356]

In [7]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## to lunch the web chrome driver
driver = webdriver.Chrome(
    executable_path="./chromedriver.exe"
    )

In [8]:
listp = ['https://www.basketball-reference.com/players/j/jacksja02.html']

In [9]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re

for url in listp:
    player_url = url
    driver.get(player_url)
    driver.execute_script("window.scrollTo(1,10000)")
    ##time.sleep(0.2)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    # Get players name
    player_name = soup.find('h1')
    player_name = player_name.find('span').text
    
    # Find all <tfoot> elements
    tfoot_elements = soup.find_all('tfoot')

    # Decompose each <tfoot> element
    for tfoot in tfoot_elements:
        tfoot.decompose()
        
    ## get players code
    parts = url.split("/")
    player_code = parts[-1].replace(".html", "")

    ## create a directory to put the players sats
    os.makedirs("All_players_stats/{}".format(player_code), exist_ok=True)
    
    ## Per Game stats
    init_per_game = soup.find_all("table", {'id':'per_game'})
    
    if init_per_game is not None and len(init_per_game) > 0:
        
        per_game = init_per_game[0]
        per_game = pd.read_html(str(per_game))[0]
        per_game = per_game[per_game['Lg'] == 'NBA']
        ##display(per_game)
        per_game.reset_index(drop=True, inplace=True)
        ##per_game.to_csv("All_players_stats/{}/per_game.csv".format(player_code))
        
    ## Per 100 Poss stats
    init_per_100_poss = soup.find_all("table", {'id':'per_poss'})
    if init_per_100_poss is not None and len(init_per_100_poss) > 0:
        
        per_100_poss = init_per_100_poss[0]
        per_100_poss = pd.read_html(str(per_100_poss))[0]
        per_100_poss = per_100_poss[per_100_poss['Lg'] == 'NBA']
        ##display(per_100_poss)
        columns_to_drop = ['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS']
        per_100_poss = per_100_poss.drop(columns=columns_to_drop)
        per_100_poss = per_100_poss.add_suffix('_100_poss')
        per_100_poss.reset_index(drop=True, inplace=True)
        ##per_100_poss.to_csv("All_players_stats/{}/per_100_poss.csv".format(player_code))

    ## Advanced stats
    init_advanced = soup.find_all("table", {'id':'advanced'})
    if init_advanced is not None and len(init_advanced) > 0:
        
        advanced = init_advanced[0]
        advanced = pd.read_html(str(advanced))[0]
        advanced = advanced[advanced['Lg'] == 'NBA']
        ##display(advanced)
        columns_to_drop = ['Season','Age', 'Tm', 'Lg', 'Pos', 'G']
        advanced = advanced.drop(columns=columns_to_drop)
        advanced = advanced.add_suffix('_advanced')
        advanced.reset_index(drop=True, inplace=True)
        
        ##advanced.to_csv("All_players_stats/{}/advanced.csv".format(player_code))

    ## concat players stats in one dataframe
    player_stats = pd.concat([per_game, per_100_poss, advanced], axis=1)
    player_stats = player_stats.loc[:, ~player_stats.columns.duplicated()]
    ## remove unnamed columns
    player_stats = player_stats.drop(columns=player_stats.columns[player_stats.columns.str.contains('Unnamed')])
    ## remove the total lines
    player_stats = player_stats[player_stats['Tm'] != 'TOT']
    player_stats = player_stats[player_stats['Tm'].str.isalpha() & player_stats['Tm'].str.isupper() & (player_stats['Tm'].str.len() == 3)]
    ## add a year column using the Season columns
    player_stats['SeasonYear'] = pd.to_datetime(player_stats['Season'], format='%Y-%y')
    player_stats = player_stats[(pd.to_datetime(player_stats['Season'], format='%Y-%y') >= pd.to_datetime('1992-93', format='%Y-%y')) & (pd.to_datetime(player_stats['Season'], format='%Y-%y') <= pd.to_datetime('2022-23', format='%Y-%y'))]
    player_stats = player_stats.drop(columns=['SeasonYear'])
    
    ##player_stats.to_csv("All_players_stats/{}/player_stats.csv".format(player_code))
    
    ##add narratives columns
    result_player = pd.DataFrame()
    dpoy_df = pd.DataFrame({'Rank': [] , 'Dpoy_votes': [], 'Dpoy_titles': [], 'First_votes': [], 'Points_won': [], 'Max_points': [], 'won_dpoy_last_season':[], 'teamates_with_dpoy':[]})
    Dpoy_votes = 0
    Dpoy_titles = 0
    
    ## go through each row in the player dataframe
    for row in player_stats.itertuples(index=False):
        player_rank = -1
        first_votes = 0
        points_won = 0
        max_points = 500
        team = row.Tm
        teamates_with_dpoy = 0
        year = int(str(row.Season).split('-')[1])
        won_dpoy_last_season = 'false'
        
        ## get the year from the season columns
        if year < 50 :
            year += 2000
        else:
            year += 1900

        ## read the DPOY stats and the team stats
        dpoy_stats = pd.read_csv("dpoy_stats/{0}.csv".format(year))
        team_stats = pd.read_csv("All_teams_stats/{0}/team_stats.csv".format(team))

        ## get each player/year stat and concat it with team/year in one row
        if year in team_stats['Year'].values:
            row_team_stats = team_stats[team_stats['Year'] == year].iloc[0]
            row_team_stats.reset_index(drop=True)
            result_player = pd.concat([result_player, row_team_stats], axis=1)
            display(result_player)

        ## check if there are team mates who won the DPOY
        for nba_team in dpoy_stats['Tm'].values:
            if nba_team == team:
                teamates_with_dpoy += 1

        if year > 1993:
            dpoy_stats_last_year = pd.read_csv("dpoy_stats/{0}.csv".format(year-1))
            players_list = dpoy_stats_last_year['Player'].values
            if player_name == players_list[0]:
                won_dpoy_last_season = 'true'
        
        ## get other info from DPOY voting tables and add them to the player (match by name)        
        if player_name in dpoy_stats['Player'].values:
            row_player_dpoy = dpoy_stats[dpoy_stats['Player'] == player_name].iloc[0]
            player_rank = row_player_dpoy['Rank']
            first_votes = row_player_dpoy['First']
            points_won = row_player_dpoy['Pts Won']
            teamates_with_dpoy -= 1
                
            ## add the number of DPOY awards a player won 
            if str(player_rank) == "1" :
                Dpoy_titles += 1

            ## increament if player recieved votes
            Dpoy_votes += 1

            dpoy_df = pd.concat([dpoy_df, pd.DataFrame({'Rank': [player_rank] , 'Dpoy_votes': [Dpoy_votes], 'Dpoy_titles': [Dpoy_titles], 'First_votes': [first_votes], 'Points_won': [points_won], 'Max_points': [max_points], 'won_dpoy_last_season':[won_dpoy_last_season],'teamates_with_dpoy':[teamates_with_dpoy]})], axis=0)
        else:
            ## if player didn't recieve votes
            dpoy_df = pd.concat([dpoy_df, pd.DataFrame({'Rank': [player_rank] , 'Dpoy_votes': [Dpoy_votes], 'Dpoy_titles': [Dpoy_titles], 'First_votes': [first_votes], 'Points_won': [points_won], 'Max_points': [max_points], 'won_dpoy_last_season':[won_dpoy_last_season],'teamates_with_dpoy':[teamates_with_dpoy]})], axis=0)
    
    result_player = result_player.T 
    player_stats.reset_index(drop=True, inplace=True)
    result_player.reset_index(drop=True, inplace=True)
    dpoy_df.reset_index(drop=True, inplace=True)

    ## concat players stats and narrative DPOY stats
    result = pd.concat([player_stats,result_player,dpoy_df], axis=1)

    ## adding players name
    result.insert(1, 'Player_name', player_name)

    ## orgnizing columns
    year_column = result.pop('Year')
    result.insert(1, 'Year', year_column)
    
    rank_column = result.pop('Rank')
    result.insert(3, 'Rank', rank_column)
    
    Dpoy_titles_column = result.pop('Dpoy_titles')
    result.insert(4, 'Dpoy_titles', Dpoy_titles_column)
    
    Dpoy_votes_column = result.pop('Dpoy_votes')
    result.insert(5, 'Dpoy_votes', Dpoy_votes_column)
    
    Dpoy_first_column = result.pop('First_votes')
    result.insert(6, 'First_votes', Dpoy_first_column)

    won_dpoy_last_season_column = result.pop('won_dpoy_last_season');
    result.insert(7,'won_dpoy_last_season', won_dpoy_last_season_column);

    teamates_with_dpoy_column = result.pop('teamates_with_dpoy');
    result.insert(8,'teamates_with_dpoy', teamates_with_dpoy_column)

    Dpoy_won_column = result.pop('Points_won')
    result.insert(9, 'Points_won', Dpoy_won_column)

    Dpoy_max_column = result.pop('Max_points')
    result.insert(10, 'Max_points', Dpoy_max_column)
    ## remove unnamed columns and save player stats
    result = result.drop(columns=result.columns[result.columns.str.contains('Unnamed')])
    result.to_csv("All_players_stats/{}/player_team_stats.csv".format(player_code))


,17
Unnamed: 0,0
Year,2019
G_team,82.0
MP_team,19880
FG_team,3113
...,...
TOV%.1_misc,13.9
DRB%_misc,77.6
FT/FGA.1_misc,0.232
Arena_misc,FedEx Forum


,17,18
Unnamed: 0,0,0
Year,2019,2020
G_team,82.0,73.0
MP_team,19880,17570
FG_team,3113,3106
...,...,...
TOV%.1_misc,13.9,12.7
DRB%_misc,77.6,77.8
FT/FGA.1_misc,0.232,0.217
Arena_misc,FedEx Forum,FedEx Forum


,17,18,19
Unnamed: 0,0,0,0
Year,2019,2020,2021
G_team,82.0,73.0,72.0
MP_team,19880,17570,17405
FG_team,3113,3106,3085
...,...,...,...
TOV%.1_misc,13.9,12.7,13.1
DRB%_misc,77.6,77.8,78.3
FT/FGA.1_misc,0.232,0.217,0.196
Arena_misc,FedEx Forum,FedEx Forum,FedEx Forum


,17,18,19,20
Unnamed: 0,0,0,0,0
Year,2019,2020,2021,2022
G_team,82.0,73.0,72.0,82.0
MP_team,19880,17570,17405,19780
FG_team,3113,3106,3085,3571
...,...,...,...,...
TOV%.1_misc,13.9,12.7,13.1,13.3
DRB%_misc,77.6,77.8,78.3,77.8
FT/FGA.1_misc,0.232,0.217,0.196,0.195
Arena_misc,FedEx Forum,FedEx Forum,FedEx Forum,FedEx Forum


,17,18,19,20,21
Unnamed: 0,0,0,0,0,0
Year,2019,2020,2021,2022,2023
G_team,82.0,73.0,72.0,82.0,82.0
MP_team,19880,17570,17405,19780,19780
FG_team,3113,3106,3085,3571,3585
...,...,...,...,...,...
TOV%.1_misc,13.9,12.7,13.1,13.3,13.1
DRB%_misc,77.6,77.8,78.3,77.8,75.9
FT/FGA.1_misc,0.232,0.217,0.196,0.195,0.206
Arena_misc,FedEx Forum,FedEx Forum,FedEx Forum,FedEx Forum,FedEx Forum


In [40]:
## concat all the csv files of players into one big dataframe
all_players_df = pd.DataFrame()
for url in All_players_urls_list:
    parts = url.split("/")
    player_code = parts[-1].replace(".html", "")
    
    player_stats = pd.read_csv("All_players_stats/{0}/player_team_stats.csv".format(player_code))
    all_players_df = pd.concat([all_players_df,player_stats], axis=0)
    all_players_df.reset_index(drop=True, inplace=True)
    
all_players_df.to_csv("All_players_stats/all_players.csv")

In [44]:
## sort according to players name
##all_players_df_sorted = all_players_df.sort_values(by='Player_name')
##all_players_df_sorted.reset_index(drop=True, inplace=True)
##all_players_df_sorted.to_csv("All_players_stats/all_players_name_sorted.csv")


## sort according to year
all_players_df_sorted = all_players_df.sort_values(by=['Year', 'Points_won'], ascending=[False, False])
all_players_df_sorted.reset_index(drop=True, inplace=True)
all_players_df_sorted = all_players_df_sorted.drop(columns=all_players_df_sorted.columns[all_players_df_sorted.columns.str.contains('Unnamed')])
## replace all the empty spaces with 0
all_players_df_sorted = all_players_df_sorted.fillna(0)
## save the all players data frame
all_players_df_sorted.to_csv("All_players_stats/all_players_year_sorted.csv")

## sort by DPOY titles/awards
#all_players_df_sorted_titles = all_players_df.sort_values(by='Dpoy_titles', ascending = False)
#all_players_df_sorted_titles.reset_index(drop=True, inplace=True)
#all_players_df_sorted_titles = all_players_df_sorted_titles.drop(columns=all_players_df_sorted_titles.columns[all_players_df_sorted_titles.columns.str.contains('Unnamed')])
#all_players_df_sorted_titles.to_csv("All_players_stats/all_players_titles_sorted.csv")